Particle Swarm Optimization and Optuna (Hyperparameter tool)

In [1]:
!pip install optuna

In [2]:
import optuna
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

# Veri örneği (verinizi kendi gerçek verinizle değiştirin)
input_data = np.random.rand(1000, 10)
labels = np.random.randint(3, size=(1000,))

# Veriyi eğitim ve test kümelerine ayırın
train_data, test_data, train_labels, test_labels = train_test_split(input_data, labels, test_size=0.2, random_state=42)


2024-03-02 20:16:09.601839: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-02 20:16:09.601945: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-02 20:16:09.750147: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Model ağı seçilip geliştirilebilir.
def create_model(trial)->Model:
    # Optimize edilecek hiperparametreleri belirle
    # Random olarak, belirlenmektedir.
    num_layers = trial.suggest_int('num_layers', 1, 32)
    num_units = trial.suggest_int('num_units', 32, 256)

    # Modeli oluştur
    # Random atanan verilerden, 10 girişli bir model belirle
    model_input = Input(shape=(10,))
    x = model_input
    for _ in range(num_layers):
        x = Dense(num_units, activation='relu')(x)
    model_output = Dense(3, activation='softmax')(x)

    model:Model = Model(inputs=model_input, outputs=model_output)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

# Farklı Optimizasyon modelleri oluşturulup seçilebilir,
def objective(trial)->float:
    # Modeli oluştur
    model:Model = create_model(trial)

    # Model üzerinde epochs değeerinin belirlenmesinde öncü ol
    epochs = trial.suggest_int('epochs', 1, 1000)

    # Modeli eğit
    model.fit(train_data, train_labels, epochs=epochs, verbose=0)

    # Modelin test kümesindeki başarımını değerlendir
    _, accuracy = model.evaluate(test_data, test_labels, verbose=0)

    return -accuracy  # Negatif çünkü Optuna en küçük değeri arar

# Optuna optimizasyonu
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)


[I 2024-03-02 20:16:20,925] A new study created in memory with name: no-name-ed4c93ef-b53a-4014-afa2-860100af1538
[I 2024-03-02 20:17:56,844] Trial 0 finished with value: -0.3100000023841858 and parameters: {'num_layers': 32, 'num_units': 111, 'epochs': 561}. Best is trial 0 with value: -0.3100000023841858.
[I 2024-03-02 20:20:16,023] Trial 1 finished with value: -0.3100000023841858 and parameters: {'num_layers': 19, 'num_units': 151, 'epochs': 968}. Best is trial 0 with value: -0.3100000023841858.
[I 2024-03-02 20:21:25,716] Trial 2 finished with value: -0.3100000023841858 and parameters: {'num_layers': 25, 'num_units': 164, 'epochs': 243}. Best is trial 0 with value: -0.3100000023841858.
[I 2024-03-02 20:23:02,836] Trial 3 finished with value: -0.33500000834465027 and parameters: {'num_layers': 10, 'num_units': 174, 'epochs': 775}. Best is trial 3 with value: -0.33500000834465027.
[I 2024-03-02 20:23:07,518] Trial 4 finished with value: -0.3400000035762787 and parameters: {'num_layer

In [4]:
# En iyi hiperparametreleri ve başarıyı görüntüle
best_params = study.best_params
best_accuracy = -study.best_value
print('En iyi hiperparametreler:', best_params)
print('En iyi başarı:', best_accuracy)

# En iyi modeli oluştur
best_model = create_model(study.best_trial)

# En iyi modeli eğit
best_model.fit(train_data, train_labels, epochs=5, verbose=1)

# En iyi modele ait veriler, pickle ile kayıt edilebilir.
# ****


# En iyi modelin başarımını kontrol et
final_accuracy = best_model.evaluate(test_data, test_labels, verbose=0)[1]
print(f'Final Model Accuracy: {final_accuracy}')

En iyi hiperparametreler: {'num_layers': 3, 'num_units': 69, 'epochs': 71}
En iyi başarı: 0.3400000035762787
Epoch 1/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.3669 - loss: 1.1008   
Epoch 2/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3458 - loss: 1.0974 
Epoch 3/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4016 - loss: 1.0912 
Epoch 4/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4061 - loss: 1.0827 
Epoch 5/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3894 - loss: 1.0878 
Final Model Accuracy: 0.32499998807907104


pickle ile parametreleri kayıt edelim.

In [5]:
import pickle
pickle.dump(best_params, open( "save.p", "wb" ))


In [6]:
file_parameter:dict=pickle.load( open( "save.p", "rb" ) )

def create_model_parameter()->Model:
    # Belirli hiperparametre seti
    num_layers =  file_parameter["num_layers"]
    num_units =  file_parameter["num_units"]

    # Modeli oluştur
    model_input = Input(shape=(10,))
    x = model_input
    for _ in range(num_layers):
        x = Dense(num_units, activation='relu')(x)
    model_output = Dense(3, activation='softmax')(x)

    model = Model(inputs=model_input, outputs=model_output)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

# Modeli oluştur
custom_model:Model = create_model_parameter()

# Modeli eğit
custom_model.fit(train_data, train_labels, epochs=file_parameter["epochs"], verbose=1)

# Modelin başarımını kontrol et
final_accuracy = custom_model.evaluate(test_data, test_labels, verbose=0)[1]
print(f'Final Model Accuracy: {final_accuracy}')


Epoch 1/71
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3544 - loss: 1.1032   
Epoch 2/71
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3952 - loss: 1.0933 
Epoch 3/71
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3716 - loss: 1.0918 
Epoch 4/71
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3698 - loss: 1.0900 
Epoch 5/71
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4091 - loss: 1.0819 
Epoch 6/71
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4613 - loss: 1.0706 
Epoch 7/71
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3924 - loss: 1.0792 
Epoch 8/71
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4205 - loss: 1.0753 
Epoch 9/71
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4333 - loss: 1.0683 
Epoch 10/71
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4605 - loss: 1.0537 
Epoch 11/71
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4682 - loss: 1.0516 
Epoch 12/71
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy